In [2]:
import os

In [3]:
%pwd

'e:\\Interest\\_Projects\\distilbart-text summarization\\Text-Summarization-NLP-Project\\research'

In [4]:
os.chdir("../")

In [12]:
%pwd

'e:\\Interest\\_Projects\\distilbart-text summarization\\Text-Summarization-NLP-Project'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [14]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


In [17]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [18]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        # Tokenize the input text (email) and target text (summary)
        input_encodings = self.tokenizer(example_batch['Email'], max_length=1024, truncation=True, padding='max_length')
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['Summary'], max_length=128, truncation=True, padding='max_length')

        # Return the tokenized data
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        # Load the dataset directly from CSV files using `load_dataset`
        dataset = load_dataset('csv', data_files={
            'train': os.path.join(self.config.data_path, 'train/train.csv'),
            'test': os.path.join(self.config.data_path, 'test/test.csv'),
            'val': os.path.join(self.config.data_path, 'val/val.csv')
        })

        # Apply tokenization to the dataset
        dataset = dataset.map(self.convert_examples_to_features, batched=True)

        # Save the transformed dataset to disk
        dataset.save_to_disk(os.path.join(self.config.root_dir, 'email_data_transformed'))

In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-11-15 22:13:40,856: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-15 22:13:40,870: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-15 22:13:40,874: INFO: common: created directory at: artifacts]
[2024-11-15 22:13:40,878: INFO: common: created directory at: artifacts/data_transformation]


Generating train split: 160 examples [00:00, 2536.51 examples/s]
Generating test split: 20 examples [00:00, 1051.12 examples/s]
Generating val split: 20 examples [00:00, 2112.52 examples/s]
Map:   0%|          | 0/160 [00:00<?, ? examples/s]e:\Interest\_Projects\distilbart-text summarization\Text-Summarization-NLP-Project\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 20/20 [00:00<00:00, 1031.96 examples/s]
